In [77]:
import mfem.ser as mfem
from mfem.common.arg_parser import ArgParser
import numpy as np
from glvis import glvis
import matplotlib.pyplot as plt
import mpi4py.MPI as MPI
print(MPI.Get_version())
import math
import h5py
import os

(4, 1)


In [92]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """
    Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """
    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0 (they will be updated)
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()
    
    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    
    return mesh

def gauss_legendre_dirs(N_dir):
    """
    Compute Gauss-Legendre quadrature points (mu_i) and weights (w_i)
    for discrete ordinates. Typically used for [-1,1].
    Returns (mu[], w[]) each of length N_dir.
    """
    mu = []
    w  = []

    gg_mu, gg_w = np.polynomial.legendre.leggauss(N_dir)
    
    mu = gg_mu.tolist()
    w  = gg_w.tolist()
    return mu, w


nx = 20
nE = 50
x_start = 0.0
x_end = 0.3
E_start = 1.0
E_end = 0.01
N_ang = 8
mu_vals, w_vals = np.polynomial.legendre.leggauss(N_ang)
order = 1
data = h5py.File("data.h5", "r")
E_vals    = data[f"E_{nE}"][:]
xs_t_vals = data[f"xs_t_{nE}"][:] 
xs_s_vals = data[f"xs_s_{nE}"][:] 
S_vals = data[f"S_{nE}"][:] 
mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()
print(dim)
inflow_value = 1.0


fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
ndofs = fes.GetVSize()
print(f"Number of finite element unknowns:: {ndofs}")
a = mfem.BilinearForm(fes)

data = h5py.File("data.h5", "r")
E_vals    = data[f"E_{nE}"][:]
xs_t_vals = data[f"xs_t_{nE}"][:] 
xs_s_vals = data[f"xs_s_{nE}"][:] 
S_vals = data[f"S_{nE}"][:] 

ess_tdof_list = mfem.intArray()
dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr.Assign(1)
dir_bdr.Size()

print(dir_bdr.Size())
print(ess_tdof_list.Size())

class TotalXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    cross-section value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(TotalXSCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]  # normalized energy coordinate in [0,1]
        # Map y ∈ [0,1] to energy E ∈ [E_start, E_end]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        # Fraction: (E - E_start)/(E_end - E_start)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])
    
class ScatteringXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the scattering cross-section Σ_s(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    scattering cross-section value. Here, y=0 corresponds to E = E_start and y=1 
    corresponds to E = E_end.
    """
    def __init__(self, xs_s_data, E_start, E_end):
        super(ScatteringXSCoefficient, self).__init__()
        self.xs_s_data = xs_s_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    S(E) value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])
    
class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    derivative value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, dS_data, E_start, E_end):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        self.dS_data = dS_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])
    
class InflowCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the inflow boundary condition.
    Returns a prescribed inflow flux value.
    """
    def __init__(self, inflow_value):
        super(InflowCoefficient, self).__init__()
        self.inflow_value = inflow_value

    def EvalValue(self, x):
        # x burada kenar üzerinde gelen noktanın koordinatlarıdır.
        return self.inflow_value
    

class QCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the source term Q(x,E).

    Maps the normalized energy coordinate (x[1] in [0,1]) to the corresponding 
    Q value. Here, y=0 corresponds to E = E_start and y=1 corresponds to E = E_end.
    
    If Q_data is given as the scalar 0, then the coefficient always returns 0,
    and E_start and E_end are not required.
    """
    def __init__(self, Q_data, E_start=None, E_end=None):
        super(QCoefficient, self).__init__()

        if isinstance(Q_data, (int, float)) and Q_data == 0:
            self.scalar_zero = True
            self.Q_data = None

        else:

            if E_start is None or E_end is None:
                raise ValueError("For non-zero Q_data, E_start and E_end must be provided.")
            self.scalar_zero = False
            if isinstance(Q_data, (int, float)):

                self.Q_data = [Q_data]
            else:
                self.Q_data = Q_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, x):
        if self.scalar_zero:
            return 0.0
        
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.Q_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))

        return float(self.Q_data[group])



inflow_coef = InflowCoefficient(inflow_value)
b = mfem.LinearForm(fes)
q_coef = QCoefficient(0)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))
b.Assemble()


2
Number of finite element unknowns:: 4000
4
0
